PEGS @ MMCW
----
Tom Killestein (t.killestein@warwick.ac.uk)

This short tutorial is a step-by-step guide to producing your own ephemeris for Sco X-1,
using a small subset of the VLT/UVES spectra from the real PEGS IV analysis.

Some links:
[PEGS IV on arXiv](https://arxiv.org/abs/2302.00018)
[VLT/UVES instrument page](https://www.eso.org/sci/facilities/paranal/instruments/uves.html)

In [ ]:
!pip install -r requirements.txt
import os, glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import least_squares

# configure JAX for use - 64-bit precision is important
from jax.config import config
import corner

config.update("jax_enable_x64", True)
import jax.numpy as jnp
from jax import jacfwd, random
import numpyro
import numpyro.distributions as dist

# astropy to handle necessary time/velocity conversions
from astropy.io import fits
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
import astropy.units as u
from astropy.coordinates import solar_system_ephemeris

solar_system_ephemeris.set("jpl")

# for progress bars
from tqdm.notebook import tqdm

# tell numpyro to use 64-bit mode
numpyro.set_host_device_count(
    8
)  # set to n_cpus - 1 ideally, although beware IO-bottlenecked at high cores.
numpyro.enable_x64()

In [ ]:
# Configure plots to have nice defaults
plt.rcParams.update({
    # "text.usetex": True,
    # "font.family": "serif",
    # "font.serif": ["Times"],
    "xtick.direction": "in",
    "xtick.top": True,
    "xtick.minor.visible": True,
    "ytick.right": True,
    "ytick.direction": "in",
    "ytick.minor.visible": True
})

Utility functions

In [ ]:
# abstracted out of scipy.optimize.minpack - prefer to use least_squares over curve_fit hence need this
# https://github.com/scipy/scipy/blob/47bb6febaa10658c72962b9615d5d5aa2513fa3a/scipy/optimize/minpack.py#L810-L816
def cov_from_jacobian(jac: jnp.array):
    _, s, VT = jnp.linalg.svd(jac, full_matrices=False)
    threshold = jnp.finfo(float).eps * max(jac.shape) * s[0]
    s = s[s > threshold]
    VT = VT[: s.size]
    pcov = jnp.dot(VT.T / s**2, VT)

    return pcov


def doppler_corr(wav: float, vel: float):
    return wav * (1 + vel / 3e8)

Define the Bowen line model

In [ ]:
def circular_orbit(t: float, pars: jnp.array):
    t0, P, gamma, K = pars
    return K * jnp.sin(2 * jnp.pi * (t - t0) / P) + gamma


def circ_orbit_resids(pars, x, y, yerr):
    return (y - circular_orbit(x, pars)) / yerr


jac_circ_orbit_resids = jacfwd(circ_orbit_resids)


def circ_orbit_chisq(pars, x, y, yerr):
    return jnp.sum(jnp.square(circ_orbit_resids(pars, x, y, yerr)))


# Bowen model - line centres taken from NIST line list.
line_centres = jnp.array(
    [
        4640.0,  # broad
        4634.13,  # NIII
        4640.64,  # NIII
        4647.42,  # CIII
        4650.25,  # CIII
        4643.386,  # OII
    ]
)

# FWHMs of broad components and emission lines (m/s)
line_fwhms = jnp.array([1250e3, 50.0e3, 50.0e3, 50.0e3, 50.0e3, 50.0e3])

# immediately convert into Gaussian sigmas
line_sigmas = line_fwhms * line_centres / 3e8 / (2 * jnp.sqrt(2 * jnp.log(2)))


def gaussian(x, a, mean, sigma_g):
    return a * jnp.exp(-((x - mean) ** 2) / (2 * sigma_g**2))


def bowen_model(
    x, params, line_centres_list=line_centres, line_sigmas_list=line_sigmas
):
    vel, bl_amp, bl_cen, nl_amp1, nl_amp2, nl_amp3, nl_amp4, nl_amp5 = params
    model = jnp.zeros_like(x)

    # manually specify, can write a nice loop later
    model = (
        gaussian(x, bl_amp, bl_cen, line_sigmas_list[0])
        + gaussian(
            x, nl_amp1, doppler_corr(line_centres_list[1], vel), line_sigmas_list[1]
        )
        + gaussian(
            x, nl_amp2, doppler_corr(line_centres_list[2], vel), line_sigmas_list[2]
        )
        + gaussian(
            x, nl_amp3, doppler_corr(line_centres_list[3], vel), line_sigmas_list[3]
        )
        + gaussian(
            x, nl_amp4, doppler_corr(line_centres_list[4], vel), line_sigmas_list[4]
        )
        + gaussian(
            x, nl_amp5, doppler_corr(line_centres_list[5], vel), line_sigmas_list[5]
        )
    )

    return model


def bowen_residual_vector(params, x, y, yerr):
    return (y - bowen_model(x, params)) / yerr


# Housekeeping to prepare the jacobians for our residual function
jac_bowen_residual_vector = jacfwd(bowen_residual_vector)

In [ ]:
# data ingest and parse
storage_loc = "data/"
framelist = glob.glob(os.path.join(storage_loc, "*.fits"))

# UVES has two arms - one red and one blue. We only want the BLUE arm for this analysis
blueframes = [
    f
    for f in framelist
    if (fits.getval(f, "HIERARCH ESO INS PATH") == "BLUE")
    & (fits.getval(f, "NCOMBINE") == 1)
]
print(f"{len(blueframes)} of {len(framelist)} frames valid - ingesting")

spectra_table = []

# The observatory and Sco X-1 don't  move (we hope!), so let's pre-compute some properties.
summary_header = fits.getheader(blueframes[0])
obsloc = (
    summary_header["HIERARCH ESO TEL GEOLON"],
    summary_header["HIERARCH ESO TEL GEOLAT"],
    summary_header["HIERARCH ESO TEL GEOELEV"],
)
obsloc = EarthLocation(*obsloc)
skyloc = SkyCoord(summary_header["RA"], summary_header["DEC"], unit="deg", frame="fk5")

for f in tqdm(blueframes, total=len(blueframes)):
    hdul = fits.open(f)
    obs_epoch = Time(hdul[0].header["DATE-OBS"])
    jd = obs_epoch.jd

    exptime = hdul[0].header["EXPTIME"]
    jd += 0.5 * exptime / 86400  # move to midpoint of exposure
    barycor = skyloc.radial_velocity_correction(
        kind="barycentric", location=obsloc, obstime=obs_epoch
    )
    snr = hdul[0].header["SNR"]

    # explicit coercion to 64-bit floats here
    wav = hdul[1].data["WAVE"].flatten().astype(float)
    flux = hdul[1].data["FLUX"].flatten().astype(float)
    flux_err = hdul[1].data["ERR"].flatten().astype(float)

    hdul.close()

    spectra_table.append(
        (f, jd, wav, flux, flux_err, barycor.to(u.m / u.s).value, snr, exptime)
    )

spectra_table = pd.DataFrame(
    spectra_table,
    columns=["filename", "jd", "wav", "flux", "flux_err", "barycor", "snr", "exptime"],
)

# Apply heliocentric/barycentric correction, but save for later so we can undo if needed
obs_epochs = Time(spectra_table.jd, format="jd", location=obsloc)
bary_tt = obs_epochs.light_travel_time(skyloc, "barycentric", obsloc)
spectra_table["barycor_dt"] = bary_tt.jd
obs_epochs += bary_tt

# overwrite times
spectra_table["jd"] = obs_epochs.utc.jd

# Coarse ephemeris estimate
init_ephemeris = [2455444.7284, 0.7873143, -113.7e3, 74.6e3]
spectra_table["expected_vel"] = (
    circular_orbit(spectra_table.jd.values, init_ephemeris)
    - spectra_table.barycor.values
)

spectra_table = spectra_table.sort_values("jd")

In [ ]:
bowen_roi = (4605, 4675)
continuum_regions = [(4605, 4615), (4666, 4675)]
plot_wav = np.linspace(*bowen_roi, 1000)

for i, spectrum in tqdm(spectra_table.iterrows(), total=len(spectra_table)):
    # build fitting masks
    wavmask = np.logical_and.reduce(
        [spectrum.wav > bowen_roi[0], spectrum.wav < bowen_roi[1]]
    )
    cont1mask = np.logical_and.reduce(
        [spectrum.wav > continuum_regions[0][0], spectrum.wav < continuum_regions[0][1]]
    )
    cont2mask = np.logical_and.reduce(
        [spectrum.wav > continuum_regions[1][0], spectrum.wav < continuum_regions[1][1]]
    )

    contmask = cont1mask | cont2mask

    # Fit a low-order Chebyshev polynomial to the continuum to remove background flux
    bg_poly = np.polynomial.Chebyshev.fit(
        spectrum.wav[contmask],
        spectrum.flux[contmask],
        w=1 / spectrum.flux_err[contmask],
        deg=3,
    )

    # ideally don't share pixels between line and continuum fitting
    fitregion_mask = ~contmask & wavmask

    detrend_wav = spectrum.wav[fitregion_mask]
    detrend_flux = spectrum.flux[fitregion_mask] - bg_poly(spectrum.wav[fitregion_mask])
    detrend_fluxerr = spectrum.flux_err[fitregion_mask]

    # Fit bounds to avoid spurious solutions
    constraints = (
        (-np.inf, 0, -np.inf, 0, 0, 0, 0, 0),
        (0, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, np.inf),
    )

    maxval = np.max(detrend_flux)

    # Some sensible initialisations for the fits
    test_pars = [
        spectrum.expected_vel,
        maxval / 2,
        4642,
        maxval,
        maxval,
        maxval,
        maxval,
        maxval,
    ]

    res = least_squares(
        bowen_residual_vector,
        test_pars,
        args=(detrend_wav, detrend_flux, detrend_fluxerr),
        bounds=constraints,
        jac=jac_bowen_residual_vector,
        method="trf",
    )

    # can recycle the optimal res.fun here since we're using normed resids
    rchsq = np.sum(np.square(res.fun)) / (len(res.fun) - len(res.x))

    cov = cov_from_jacobian(res.jac)
    res_errs = np.sqrt(np.diag(cov) * rchsq)

    # Let's save our fitted velocity and errors, and apply our barycentric corrections
    # here
    spectra_table.at[i, "vel"] = res.x[0] + spectrum.barycor
    spectra_table.at[i, "vel_err"] = np.sqrt(
        res_errs[0] ** 2 + 200**2
    )  # ensure we get non-zero residuals, ~= systematic

    # As a diagnostic, plot every 5th spectrum to check things are working as expected
    if not (i % 5):
        plt.plot(spectrum.wav, spectrum.flux - bg_poly(spectrum.wav))
        plt.plot(plot_wav, bowen_model(plot_wav, res.x))
        plt.xlim(bowen_roi[0] - 10, bowen_roi[1] + 10)
        plt.axvline(bowen_roi[0], ls="--", c="k")
        plt.axvline(bowen_roi[1], ls="--", c="k")
        plt.xlabel("Wavelength $(\AA)$")
        plt.ylabel("Flux")
        plt.ylim(-20, spectrum.flux[wavmask].max() + 50 - spectrum.flux[wavmask].min())
        plt.axhline(0, c="k", ls="--")

        for c in continuum_regions:
            plt.fill_betweenx(
                [0, 1200], [c[0], c[0]], [c[1], c[1]], color="r", alpha=0.2
            )

        # plt.plot(plot_wav, bowen_model(plot_wav, test_pars))
        plt.show()

Fitting complete!
---
We now have the key data needed for the ephemeris - radial velocities of the donor in Sco X-1, and estimates of uncertainty.

In [ ]:
spectra_table

In [ ]:
plt.errorbar(spectra_table.jd.values, spectra_table.vel.values, yerr=spectra_table.vel_err.values, fmt='.k')
plt.xlabel("Time (JD)")
plt.ylabel("Velocity (m/s)")

Determining the orbital parameters and ephemeris
====

In [ ]:
# local tweak to ensure we're in the right region of parameter space
local_polish = least_squares(
    circ_orbit_resids,
    x0=[2455444.7284, 0.7873143, -113.7e3, 74.6e3],
    args=(
        spectra_table.jd.values,
        spectra_table.vel.values,
        spectra_table.vel_err.values
    ),
    jac=jac_circ_orbit_resids,
    method="trf",
    x_scale="jac",
)

# Phase on the best-fit ephemeris
t0_init, P_init, gam_init, K_init = local_polish.x
init_phase = ((spectra_table.jd.values - t0_init) / P_init) % 1

times_plot = np.linspace(spectra_table.jd.min(), spectra_table.jd.max(), 10000)
plt.errorbar(spectra_table.jd, spectra_table.vel, spectra_table.vel_err, fmt='.k')
plt.plot(times_plot, circular_orbit(times_plot, local_polish.x))
plt.show()

plt.errorbar(init_phase, spectra_table.vel, yerr=spectra_table.vel_err, fmt='.k')
plt.xlabel("Phase")
plt.ylabel("Velocity (m/s")
plot_phase = np.linspace(0, 1, 1000)
folded_model = circular_orbit(plot_phase, [0, 1, gam_init, K_init])
plt.plot(plot_phase, folded_model)
plt.show()

plt.hist(local_polish.fun, bins="auto")
plt.xlabel("Normalised residual")
plt.ylabel("N")
plt.show()
print(local_polish.x)

alldata_ephem = local_polish.x
print(
    np.sum(np.square(local_polish.fun)) / (len(local_polish.fun) - len(local_polish.x))
)

Parameter inference with MCMC
-----


In [ ]:
# To avoid multimodality:
tmin = t0_init - 3 * P_init
tmax = t0_init + 3 * P_init

# Definie the probabilistic model + priors
def joint_circular_orbit_generative_model(x, y, yerr):
    T0 = numpyro.sample("T0", dist.Uniform(tmin, tmax))
    P = numpyro.sample("P", dist.Uniform(0.7, 0.8))
    gamma = numpyro.sample("gamma", dist.Uniform(-200e3, 0))
    K = numpyro.sample("K", dist.Uniform(0, 100e3))

    efac = numpyro.sample(f"efac", dist.Uniform(0.5, 10))

    pars = [T0, P, gamma, K]
    model = circular_orbit(x, pars)

    obs = numpyro.sample("obs", dist.Normal(model, yerr*efac), obs=y)
    return obs

In [ ]:
rng_key = random.PRNGKey(0)
rng_key, _rng_key = random.split(rng_key)

# start in right region of parameter space
paramnames = ["T0", "P", "gamma", "K"]
init_location = {n: v for n, v in zip(paramnames, local_polish.x)}


init_strategy = numpyro.infer.init_to_value(values=init_location)

kernel = numpyro.infer.NUTS(
    joint_circular_orbit_generative_model, init_strategy=init_strategy, dense_mass=True
)

mcmc = numpyro.infer.MCMC(
    kernel,
    num_warmup=3000,
    num_samples=1000,
    chain_method="parallel",
    num_chains=1,
    jit_model_args=True,
)

mcmc.run(
    rng_key=rng_key,
    x=spectra_table.jd.values,
    y=spectra_table.vel.values,
    yerr=spectra_table.vel_err.values,
    extra_fields=[
        "energy",
        "potential_energy",
        "diverging",
        "accept_prob",
        "num_steps",
    ],
)

mcmc.print_summary()

In [ ]:
samples = mcmc.get_samples()

# trace plots
fig, ax = plt.subplots(len(samples), 1, dpi=120, sharex=True)
plt.subplots_adjust(hspace=0)

for i, (name, s) in enumerate(samples.items()):
    ax[i].plot(s)
    ax[i].set_ylabel(name.replace("_", "\_"))

ax[-1].set_xlabel("Steps")
plt.show()

chainwise_samples = mcmc.get_samples(group_by_chain=True)
chainwise_diag_fields = mcmc.get_extra_fields(group_by_chain=True)

chain_marginal_energy = chainwise_diag_fields["energy"]
e_bfmi = np.mean(np.square(np.diff(chain_marginal_energy, axis=1)), axis=1) / np.var(
    chain_marginal_energy, axis=1, ddof=1
)
print(e_bfmi)
dE = np.diff(chain_marginal_energy, axis=1) - np.diff(
    chain_marginal_energy, axis=1
).mean(axis=1, keepdims=True)
E = chain_marginal_energy - np.mean(chain_marginal_energy, axis=1, keepdims=True)

fig = plt.figure(dpi=120)
fig.patch.set_facecolor("white")

plt.hist(dE.flatten(), bins=30, label="Transition energy", range=(-20, 20))
plt.hist(E.flatten(), bins=30, label="Marginal energy", range=(-20, 20), alpha=0.5)
plt.xlabel("$E$ - $<E>$")
plt.legend(edgecolor=None)
plt.show()

In [ ]:
samples_name_mapper = {
    "K": "$K$",
    "P": "$P$",
    "T0": "$T_0$",
    "efac": "$\varepsilon_\mathrm{VLT}$",
    "gamma": "$\gamma$",
}

test_plot_samples =    {
        samples_name_mapper[key]: value[0][:1000]
        for key, value in chainwise_samples.items()
    }
samples_rectified = {samples_name_mapper[key]: value for key, value in samples.items()}

fig = plt.figure(dpi=300, figsize=(7.05556, 1.2 * 7.05556), clear=True)

fig = corner.corner(
    test_plot_samples,
    use_math_text=True,
    color="C0",
    label_kwargs=dict(fontsize=10),
    max_n_ticks=3,
    labelpad=0.1,
    fig=fig,
)
plt.show()

Minimising covariance between $T_0$ and $P$
-----

In [ ]:
ncycles = np.arange(-2000, 2000)[:, np.newaxis]

T_n = samples['T0'] + ncycles*samples['P']

cycle_offset_corr = [np.abs(np.corrcoef(T, samples['P'])[0][1]) for T in T_n]

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=120)
fig.patch.set_facecolor('white')
ax.plot(ncycles, cycle_offset_corr, label='R')
# ax.plot(ncycles, [np.abs(np.cov(T, samples['P'])[0][1]) for T in T_n], label="cov($T_0, P$)")
plt.yscale("log")
plt.xlabel("Cycles from least-squares $T_0$")
plt.ylabel("Magnitude")
plt.legend(frameon=False)

In [ ]:
T0_new_samples = T_n[np.argmin(cycle_offset_corr)]
Tasc_new_samples = T0_new_samples - 0.25*samples['P']

In [ ]:
plt.hist(Tasc_new_samples, bins='auto', label=f"RMS: {np.std(Tasc_new_samples)*86400:.2f}s")
plt.title("$T_{asc}$ from PEGS workshop")
plt.legend(frameon=False)
plt.show()

plt.hist(samples['P']*86400, bins='auto', label=f"RMS: {np.std(samples['P'])*86400:.2f}s")
plt.legend(frameon=False)
plt.show()